In [2]:
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
# Прописываем конвертер координат для парсинга из Википедии
def lat_lon_convertor(xy, i):
    xy = xy.split()[::3]
    x = re.split("°|′|″", xy[0])[:3]
    x = float(x[0])+float(x[1])/60+float(x[2])/3600
    y = re.split("°|′|″", xy[1])[:3]
    y = float(y[0])+float(y[1])/60+float(y[2])/3600
    return [x, y][i]

# Прописываем код для загрузки иконки линии метро
# (изначально планировалось написать графический интерфейс, но не хватило времени)
def image_download(href, name):
    adress = "subway markers/" + name + ".png"
    with open(adress, "wb") as img:
        img.write(requests.get("http:"+href).content)
    return adress

In [ ]:
station = []
line = []
h = []
lat = []
lon = []
img = []
district = []

# Парсим список станций метро СПБ из Википедии.
page = requests.get("https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D1%81%D1%82%D0%B0%D0%BD%D1%86%D0%B8%D0%B9_%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3%D1%81%D0%BA%D0%BE%D0%B3%D0%BE_%D0%BC%D0%B5%D1%82%D1%80%D0%BE%D0%BF%D0%BE%D0%BB%D0%B8%D1%82%D0%B5%D0%BD%D0%B0")
soup = BeautifulSoup(page.text, "html.parser")
for i, row in enumerate(soup.find("table").findAll("tr")[1:]):
    station.append(row.findAll("td")[1].find("a").text)
    line.append(row.findAll("td")[0].find("a")["title"])
    h.append(int(row.findAll("td")[4].text))
    lon.append(lat_lon_convertor(row.findAll("td")[5].find("a").text, 0))
    lat.append(lat_lon_convertor(row.findAll("td")[5].find("a").text, 1))
    img.append(image_download(row.findAll("td")[0].find("a").find("img")["src"], line[i]))
    district.append(row.findAll("td")[6].text)

In [ ]:
import pandas as pd
import geopandas as gpd

# На основе полученных данных строим GeoDataFarame
stations_df = pd.DataFrame({
    "station":station,
    "line":line,
    "depth":h,
    "img":img,
    "lat":lat,
    "lon":lon,
    "district":district
})
stations_df.insert(2,column="time",value=None)
stations_gdf = gpd.GeoDataFrame(stations_df, geometry=gpd.points_from_xy(stations_df.lat, stations_df.lon))

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep
# Импортируем Selenium

In [ ]:
'''Парсим маршруты в Яндекс.Метро по списку станций из Википедии. В качестве облегчения задачи и ускорения решения
парсим не матрицу "пункт-назначение" для каждого элемента, а выбираем "центральный пункт" системы, до которого
строим маршруты. Обрабатываем исключения в виде маршрутов до самих себя и закрытых на ремонт станций, от которых
построение маршрута невозможно'''

service = Service("C:/Users/user/py/chromedriver/chromedriver.exe")
driver = webdriver.Chrome(service=service)
driver.get("https://yandex.ru/metro/saint-petersburg?scheme_id=sc60983525")
sleep(2)
time = []
center_point = "Невский проспект"
for station in stations_df["station"]:
    if station == center_point:
        time.append("0 мин")
    else:
        sleep(1)
        start_point = [el for el in driver.find_elements(By.XPATH, "//input")][0]
        sleep(1)
        start_point.send_keys(station)
        sleep(1)
        start_point.send_keys(Keys.ENTER) 
        sleep(1)
        end_point = [el for el in driver.find_elements(By.XPATH, "//input")][1]
        sleep(1)
        end_point.send_keys(center_point)
        sleep(1)
        end_point.send_keys(Keys.ENTER)
        try:
            time.append(sorted([el.text for el in driver.find_elements(By.CLASS_NAME, "masstransit-route-snippet-view__route-duration")])[0])
        except IndexError:
            time.append("? мин")
        sleep(1)
        start_point.click()
        start_point.send_keys(Keys.CONTROL+"a")
        start_point.send_keys(Keys.DELETE)
        sleep(1)
        end_point.click()
        end_point.send_keys(Keys.CONTROL+"a")
        end_point.send_keys(Keys.DELETE)



In [ ]:
# На данном этапе переводим записи формата "Х мин." в целочисленный тип
# Считаем время в пути от закрытой станции за среднее арифметическое путей от её соседей.
from statistics import mean
try:
    time = [re.sub(" мин", "", t) for t in time]
except TypeError:
    pass
time_copy = []
for i, t in enumerate(time):
    if t != "?":
        time_copy.append(int(t))
    else:
        converted = round(mean([int(time[i-1]),int(time[i+1])]))
        time_copy.append(converted)
time = time_copy
stations_df["time"] = time

# На основании глубины залегания станции делаем примерную оценку времени входа-выхода для уточнения значения пути до центральной точки системы.
try:
    stations_df.insert(3,column="entrance",value=None)
except:
    stations_df["entrance"] = None
stations_df["district"] = stations_df["district"].apply(lambda x: re.sub(r"\n", "", x))
stations_df["entrance"] = pd.cut(stations_df["depth"], bins=[-1, 0, 20, 90], labels=[2, 3, 5]).fillna(2)
stations_df.to_excel("subway.xlsx")
stations_gdf = gpd.GeoDataFrame(stations_df, geometry=gpd.points_from_xy(stations_df.lat, stations_df.lon)).drop(columns='entrance')
stations_gdf.to_file("stations.gpkg")

In [46]:
stations_df

,station,line,time,entrance,depth,img,lat,lon,district
0,Девяткино,Кировско-Выборгская линия,39,2,0,subway markers/Кировско-Выборгская линия.png,30.442222,60.050278,Всеволожский район Ленинградской области
1,Гражданский проспект,Кировско-Выборгская линия,35,5,64,subway markers/Кировско-Выборгская линия.png,30.418056,60.035000,Калининский район
2,Академическая,Кировско-Выборгская линия,32,5,64,subway markers/Кировско-Выборгская линия.png,30.395833,60.012500,Калининский район
3,Политехническая,Кировско-Выборгская линия,29,5,65,subway markers/Кировско-Выборгская линия.png,30.370833,60.008611,Калининский район
4,Площадь Мужества,Кировско-Выборгская линия,27,5,67,subway markers/Кировско-Выборгская линия.png,30.366667,59.999722,Калининский район
...,...,...,...,...,...,...,...,...,...
67,Бухарестская,Фрунзенско-Приморская линия,20,5,65,subway markers/Фрунзенско-Приморская линия.png,30.368611,59.883889,Фрунзенский район
68,Международная,Фрунзенско-Приморская линия,22,5,65,subway markers/Фрунзенско-Приморская линия.png,30.378889,59.870278,Фрунзенский район
69,Проспект Славы,Фрунзенско-Приморская линия,24,5,59,subway markers/Фрунзенско-Приморская линия.png,30.394722,59.856944,Фрунзенский район
70,Дунайская,Фрунзенско-Приморская линия,27,3,17,subway markers/Фрунзенско-Приморская линия.png,30.411111,59.839722,Фрунзенский район
